In [1]:
from qcodes.instrument_drivers.tektronix.Keithley_s46 import S46

In [2]:
s46 = S46("s2", "GPIB0::7::INSTR")

Connected to: KEITHLEY INSTRUMENTS INC. SYSTEM 46 (serial:1327388, firmware:A03) in 0.36s


In [13]:
s46.ask("*IDN?")

'KEITHLEY INSTRUMENTS INC.,MODEL SYSTEM 46, 1327388, A03  '

In [14]:
s46.ask(":CONF:CPOL?")

'6,6,6,6,0,0,0,0,1,0,0,1'

In [15]:
s46.ask(":CLOS?")

'(@1,7)'

In [3]:
len(s46.channels)

26

In [4]:
for channel in s46.channels: 
    print(f"{channel.short_name} is on relay {channel.relay_id}")

channel1 is on relay A
channel2 is on relay A
channel3 is on relay A
channel4 is on relay A
channel5 is on relay A
channel6 is on relay A
channel7 is on relay B
channel8 is on relay B
channel9 is on relay B
channel10 is on relay B
channel11 is on relay B
channel12 is on relay B
channel13 is on relay C
channel14 is on relay C
channel15 is on relay C
channel16 is on relay C
channel17 is on relay C
channel18 is on relay C
channel19 is on relay D
channel20 is on relay D
channel21 is on relay D
channel22 is on relay D
channel23 is on relay D
channel24 is on relay D
channel25 is on relay 5
channel26 is on relay 8


In [5]:
s46.channels[0].state()

'open'

In [6]:
s46.channels[0].state("close")

In [10]:
s46.channels[0].state()

'close'

In [11]:
try: 
    s46.channels[1].state("close")
    raise Exception("We should not be here ")
except RuntimeError as e: 
    print(e)

('Relay already in use by channel 1', 'setting s2_channel2_state to close')


In [12]:
s46.channels[1].state("open")  # This should have no effect

In [13]:
try: 
    s46.channels[5].state("close")
    raise Exception("We should not be here ")
except RuntimeError as e: 
    print(e)

('Relay already in use by channel 1', 'setting s2_channel6_state to close')


In [7]:
s46.channels[6].state("close")

In [15]:
try: 
    s46.channels[7].state("close")
    raise Exception("We should not be here ")
except RuntimeError as e: 
    print(e)

('Relay already in use by channel 7', 'setting s2_channel8_state to close')


In [16]:
s46.channels[0].state("open")

In [17]:
s46.channels[1].state("close")

In [18]:
try: 
    s46.channels[0].state("close")
    raise Exception("We should not be here ")
except RuntimeError as e: 
    print(e)

('Relay already in use by channel 2', 'setting s2_channel1_state to close')
